In [30]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline
import torch.nn as nn


torch.manual_seed(1337)

In [27]:
with open('datasets/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()


In [31]:
clean = sorted(set(''.join(text)))
vocab_size = len(clean)
stoi = { ch:i for i,ch in enumerate(clean) }
itos = { i:ch for i,ch in enumerate(clean) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [38]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline
import torch.nn as nn


torch.manual_seed(1337)

#hyper parameter
batch_size = 4
block_size = 8
n_embd = 32
dropout = 0.2
n_layers = 4
num_heads = 4
learning_rate = 1e-3
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

with open('datasets/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

clean = sorted(set(''.join(text)))
vocab_size = len(clean)
stoi = { ch:i for i,ch in enumerate(clean) }
itos = { i:ch for i,ch in enumerate(clean) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    idx = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in idx])
    y = torch.stack([data[i+1:i+block_size+1] for i in idx])
    return x,y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() 
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters, device=device)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.head_size = head_size
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self,x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        
        wei = q @ k.transpose(-2,-1) * self.head_size **-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        return wei @ v
    
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads):
        super().__init__()
        head_size = n_embd // num_heads
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out)) 
        return out

class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.ffd = nn.Sequential(
            nn.Linear(n_embd, n_embd * 4),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
           nn.Dropout(dropout)
        )
    def forward(self,x):
        return self.ffd(x)

class Block(nn.Module):
    def __init__(self, num_heads):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.sa = MultiHeadAttention(num_heads)
        self.ln2 = nn.LayerNorm(n_embd)
        self.FeedForward = FeedForward()
        
    def forward(self,x):
        x = x + self.sa(self.ln1(x))
        x = x + self.FeedForward(self.ln2(x))
        return x
    
class GPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[
            Block(num_heads) for _ in range(n_layers)
        ])
        self.final_ln = nn.LayerNorm(n_embd)
        self.final_linear = nn.Linear(n_embd, vocab_size)
        
    def forward(self, x, targets=None):
        # x is a tensor of shape (B, T), where B is the batch size and T is the sequence length
        B, T = x.shape
        
        # tok_embd should have shape (B, T, n_embd)
        tok_embd = self.token_embedding_table(x)  # (B, T, n_embd)
        pos_embd = self.position_embedding_table(torch.arange(T, device=x.device))  # (T, n_embd)
        x = tok_embd + pos_embd #B, T, n_embd
        x = self.blocks(x)
        logits = self.final_linear(self.final_ln(x)) #(B, T, vocab_size)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, x, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = x[:, -block_size:] #last 8 tokens only if more that 8..
            logits, loss = self(idx_cond) # logits: (B, T, vocab_size) → (1, 1, 65) at first
            logits = logits[:, -1, :] # focus only on the last time step # becomes (1,65)
            probs = F.softmax(logits, dim=-1) # shape: (1, 65)
            idx_next = torch.multinomial(probs, num_samples=1) # shape: (1, 1)
            x = torch.cat((x, idx_next), dim=1) # shape grows → (1, 2), (1, 3), ..., (1, T+max_new_tokens)
        return x
        
        
model = GPT().to(device)

# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
start = torch.zeros((1,1), dtype=torch.long, device=device)  # BOS = index 0 char if that's ' '
out_idx = model.generate(start, max_new_tokens=200)
print(decode(out_idx[0].tolist()))
        

0.054977 M parameters
step 0: train loss 4.2215, val loss 4.2277
step 100: train loss 3.1598, val loss 3.2041
step 200: train loss 2.9248, val loss 2.9942
step 300: train loss 2.7915, val loss 2.8178
step 400: train loss 2.7588, val loss 2.7693
step 500: train loss 2.6318, val loss 2.6854
step 600: train loss 2.5813, val loss 2.5877
step 700: train loss 2.5607, val loss 2.5921
step 800: train loss 2.5678, val loss 2.5739
step 900: train loss 2.5434, val loss 2.5625
step 1000: train loss 2.5068, val loss 2.5376
step 1100: train loss 2.4844, val loss 2.4988
step 1200: train loss 2.4898, val loss 2.5026
step 1300: train loss 2.5150, val loss 2.4933
step 1400: train loss 2.4740, val loss 2.4913
step 1500: train loss 2.4553, val loss 2.4668
step 1600: train loss 2.5012, val loss 2.4856


KeyboardInterrupt: 